**by Claude**

##  AutoGen AI Agents Example
Here's a simple Python application with two different AutoGen agents communicating with each other:

In [1]:
import autogen
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

import os
from dotenv import load_dotenv

### SETUP

In [2]:
# Load environment variables 
load_dotenv(override = True)
open_api_key = os.getenv('OPENAI_API_KEY')

# Set environment variables
# os.environ['OPENAI_API_KEY'] = open_api_key

MODEL_GPT = 'gpt-4o-mini'

### Configure OpenAI API

In [3]:
# Configure OpenAI API

api_configuration = {
    "model": MODEL_GPT,
    "api_key": open_api_key,
}

### Create a configuration list

In [4]:
# Create a configuration list

config_list = [api_configuration]

### Configure the agents

In [5]:
# Configure the agents

assistant_config = {
    "seed": 42,  # For reproducibility
    "config_list": config_list,
    "temperature": 0.7,
}

### Create an assistant agent that can generate code and explain concepts

In [6]:
# Create an assistant agent that can generate code and explain concepts

# assistant = AssistantAgent(
#     name="Assistant",
#     system_message="You are a helpful AI assistant that specializes in Python programming. You provide clear, concise code with explanations.",
#     llm_config=assistant_config,
# )

# Create an assistant agent that can generate code and explain concepts
# Add termination message

assistant = AssistantAgent(
    name="Assistant",
    system_message="""You are a helpful AI assistant that specializes in Python programming. 
    You provide clear, concise code with explanations. 
    After you've completed the task, end with the message 'TASK COMPLETED'.""",
    llm_config=assistant_config,
)

### Create a code executor agent that can run and test code

In [7]:
# Create a code executor agent that can run and test code

# executor = UserProxyAgent(
#     name="Executor",
#     system_message="You are a code execution agent. You run code provided by the Assistant and report results or errors.",
#     human_input_mode="NEVER",  # Don't ask for human input
#     code_execution_config={"work_dir": "coding", "use_docker": False},
# )

# Create a code executor agent that can run and test code

executor = UserProxyAgent(
    name="Executor",
    system_message="""You are a code execution agent. You run code provided by the Assistant and report results or errors.
    When you see 'TASK COMPLETED' from the Assistant and the code works correctly, respond with 'CONVERSATION FINISHED' and stop.""",
    human_input_mode="NEVER",  # Don't ask for human input
    code_execution_config={"work_dir": "coding", "use_docker": False},
)

### Explicitly register reply functions with termination conditions

In [8]:
# Explicitly register reply functions with termination conditions

def executor_reply(recipient, messages, sender, config):
    """Function to determine if the executor should terminate the conversation"""
    if "TASK COMPLETED" in messages[-1]["content"]:
        # Check if the executed code ran successfully before terminating
        return "CONVERSATION FINISHED"
    return False  # Continue the conversation

### Register the custom reply function

In [9]:
# Register the custom reply function

executor.register_reply(
    [assistant],
    executor_reply,
    config={"callback": None}
)

### Define how the agents will interact with a max_turns parameter

In [10]:
# Define how the agents will interact

# def start_task(task_description):
#     # Executor initiates the conversation
#     executor.initiate_chat(
#         assistant,
#         message=f"I need help with the following task: {task_description}. Please provide Python code to accomplish this."
#     )

# Define how the agents will interact with a max_turns parameter

def start_task(task_description, max_turns=10):
    # Executor initiates the conversation with a maximum number of turns
    executor.initiate_chat(
        assistant,
        message=f"I need help with the following task: {task_description}. Please provide Python code to accomplish this.",
        max_turns=max_turns
    )

### Example usage

In [11]:
# Example usage

# Define a sample task
task = "Create a simple data analysis script that calculates basic statistics (mean, median, mode) for a list of numbers."

# Start the interaction
# start_task(task)

# Start the interaction with a maximum of 5 turns
start_task(task, max_turns=5)

# You can modify the task description to suit your needs
# For example:
# task = "Build a web scraper to extract headlines from a news website"
# start_task(task, max_turns=5)

Executor (to Assistant):

I need help with the following task: Create a simple data analysis script that calculates basic statistics (mean, median, mode) for a list of numbers.. Please provide Python code to accomplish this.

--------------------------------------------------------------------------------
Assistant (to Executor):

Certainly! Below is a simple Python script that calculates the mean, median, and mode for a list of numbers using the `statistics` module.

```python
import statistics

def calculate_statistics(numbers):
    if not numbers:
        return "The list is empty."

    mean = statistics.mean(numbers)
    median = statistics.median(numbers)
    try:
        mode = statistics.mode(numbers)
    except statistics.StatisticsError:
        mode = "No unique mode found"

    return {
        'Mean': mean,
        'Median': median,
        'Mode': mode
    }

# Example usage
numbers = [1, 2, 2, 3, 4, 4, 4, 5]
stats = calculate_statistics(numbers)

print(stats)
```

### Ex

ValueError: too many values to unpack (expected 2)